In [1]:
import pandas as pd 
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Reshape, Conv2D, MaxPooling2D, BatchNormalization, Dropout
import numpy as np
import cv2 
import os
from sklearn.model_selection import train_test_split 

In [2]:
image_size = (104,104)
batch_size = 32

In [3]:
def сorrecting_data():
    x = []
    y = []
    boxes = []
    df = pd.read_csv('_annotations.csv') 
    df['xmin'] = df['xmin'].apply(lambda x: x // 4)
    df['ymin'] = df['ymin'].apply(lambda x: x // 4)
    df['xmax'] = df['xmax'].apply(lambda x: x // 4)
    df['ymax'] = df['ymax'].apply(lambda x: x // 4)
    df['width'] = df['width'].apply(lambda x: x // 4)
    df['height'] = df['height'].apply(lambda x: x // 4)
    h = df['height']
    w = df['width']
    for x1,y1,x2,y2 in zip(df['xmin'], list(df['ymin']), 
                       list(df['xmax']), list(df['ymax'])):
        arr = [x1, y1, x2, y2]
        boxes.append(arr)
    return df, boxes

In [4]:
def data_metrics(df,boxes):
    images = []
    coordinates = []
    
    for index, boxes in df.iterrows():
        image = cv2.imread(index)
        images.append(image)
        coordinates.append(boxes)
    return images, coordinates

In [5]:
# Масштабирование и преобразование извлеченных областей
def data(images, coordinates, image_size):
    scaled_regions = []
    labels = []
    for image, coords in zip(images, coordinates):
        #if x != y != w != h != 0
        x, y, w, h = coords
        car_region = image[y:y+h, x:x+w]
        scaled_region = cv2.resize(car_region, image_size)
        scaled_regions.append(scaled_region)
        labels.append(1)  # Предполагая, что все извлеченные области - это машины
    # Преобразование входных данных в массивы NumPy
    scaled_regions = np.array(scaled_regions)
    labels = np.array(labels)
    return scaled_regions, labels


In [6]:
def data_trash_preparation(df):
    images = []
    
    for index, boxes in df.iterrows():
        image = cv2.imread(index)
        images.append(image)
    return images

In [7]:
def data_trash(images, image_size):
    scaled_regions = []
    labels = []
    for image in images:
        scaled_region = cv2.resize(image, image_size)
        scaled_regions.append(scaled_region)
        labels.append(0)  # Предполагая, что все извлеченные области - это машины
    # Преобразование входных данных в массивы NumPy
    scaled_regions = np.array(scaled_regions)
    labels = np.array(labels)
    return scaled_regions, labels

In [45]:
df_trash = pd.DataFrame({'filename': os.listdir('DataSet_cars_train\\1_trash')})
df_trash = df_trash.set_index('filename')
df_trash = df_trash.astype(str)

In [46]:
%cd DataSet_cars_train/train_1
df_train, boxes_train = сorrecting_data()
df_train = df_train.set_index('filename')
#df = df_train.astype(str)
image_train, image_coords = data_metrics(df_train, boxes_train)
%cd ../..
# %cd DataSet_cars_valid/valid
# df_valid, boxes_valid = сorrecting_data()
# df_valid = df_valid.set_index('filename')
# # df = df_valid.astype(str)
# image_valid, image_coords = data_metrics(df_valid, boxes_valid)
# %cd ../..

C:\Users\Matvey\Проект\Project\DataSet_cars_train\train_1
C:\Users\Matvey\Проект\Project


In [66]:
len(image_train)

27241

In [10]:
del df_train

In [11]:
%cd DataSet_cars_train/1_trash
image_trash = data_trash_preparation(df_trash)
%cd ../..

C:\Users\Matvey\Проект\Project\DataSet_cars_train\1_trash
C:\Users\Matvey\Проект\Project


In [12]:
del df_trash

In [13]:
region_train, car_train = data(image_train,boxes_train, image_size)
del image_train
del boxes_train
region_trash, trash_train = data_trash(image_trash, image_size)
del image_trash
# region_valid, car_valid = data(image_valid,boxes_valid, image_size)

In [14]:
model = keras.Sequential([
    Conv2D(128, (3, 3), activation='relu', input_shape=(104, 104, 3)),  # свёрточный слой
    #попробуй по 2-3 свёрточных слоя сделать
    MaxPooling2D((2, 2), strides=2),  # уменьшение карт признаков
    Dropout(0.2),  # отключение 20 процентов нейронов

#     Conv2D(128, (3, 3), activation='relu'),  # свёрточный слой
#     MaxPooling2D((2, 2), strides=2),  # уменьшение карт признаков
#     Dropout(0.2),  # отключение 20 процентов нейронов

    Conv2D(256, (3, 3), activation='relu'),  # свёрточный слой
    MaxPooling2D((2, 2), strides=2),  # уменьшение карт признаков
    Dropout(0.2),  # отключение 20 процентов нейронов

#     Conv2D(256, (3, 3), activation='relu'),  # свёрточный слой
#     MaxPooling2D((2, 2), strides=2),  # уменьшеи карт признаков
#     Dropout(0.2),  # отключение 20 процентов нейронов

    Conv2D(512, (3, 3), activation='relu'),  # свёрточный слой
    MaxPooling2D((2, 2), strides=2),  # уменьшение карт признаков
    Dropout(0.2),  # отключение 20 процентов нейронов

#     Conv2D(512, (3, 3), activation='relu'),  # свёрточный слой
#     MaxPooling2D((2, 2), strides=2),  # уменьшение карт признаков
#     Dropout(0.2),  # отключение 20 процентов нейронов

    Flatten(),
    Dense(512, activation='relu'),  # обычные нейроны
    Dropout(0.2),  # отключение 20 процентов нейронов
    Dense(256, activation='relu'),  # обычные нейроны
    Dropout(0.2),  # отключение 20 процентов нейронов
    Dense(128, activation='relu'),  # обычные нейроны
    Dropout(0.2),  # отключение 20 процентов нейронов
    Dense(1, activation='sigmoid'),  # выходной слой
])


In [15]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [40]:
train_data, val_data, train_labels, val_labels = train_test_split(
    region_train, car_train, test_size=0.2, random_state=42)

In [33]:
train_labels

[array([0, 0, 0, ..., 0, 0, 0])]

In [41]:
train_data_trash, val_data_trash, train_labels_trash, val_labels_trash = train_test_split(
    region_trash, trash_train, test_size=0.1, random_state=42)

In [44]:
history = model.fit((train_data, train_data_trash), train_labels ,epochs = 10, batch_size = batch_size, validation_data = (val_data, val_labels))

ValueError: Data cardinality is ambiguous:
  x sizes: 21792, 10135
  y sizes: 21792
Make sure all arrays contain the same number of samples.

In [17]:
model.save('rofl.h5')

In [25]:
del region_valid
del region_train
del car_valid
del car_train